In [22]:
import pandas as pd
import nlp
import numpy as np
import torch

In [23]:
datasets = {
    "cw-train": nlp.load_dataset("csv", name="cw-train", data_files="data/checkworthy/english_train.tsv", delimiter="\t"),
    "cw-dev": nlp.load_dataset("csv", name="cw-dev", data_files="data/checkworthy/english_dev.tsv", delimiter="\t"),
    "cw-dev-test": nlp.load_dataset("csv", name="cw-dev-test", data_files="data/checkworthy/english_dev-test.tsv", delimiter="\t"),
}

Using custom data configuration cw-train
Using custom data configuration cw-dev
Using custom data configuration cw-dev-test


In [24]:
for task_name, dataset in datasets.items():
    print(task_name)
    print(datasets[task_name]["train"][1])
    print()

cw-train
{'Sentence_id': 19099, 'Text': "Now, let's balance the budget and protect Medicare, Medicaid, education and the environment.", 'class_label': 'No'}

cw-dev
{'Sentence_id': 80, 'Text': "We're consuming 50 percent of the world's cocaine.", 'class_label': 'Yes'}

cw-dev-test
{'Sentence_id': 37440, 'Text': "There's no way they would give it up.", 'class_label': 'No'}



In [25]:
#https://colab.research.google.com/github/zphang/zphang.github.io/blob/master/files/notebooks/Multi_task_Training_with_Transformers_NLP.ipynb#scrollTo=aVX5hFlzmLka
import transformers
import torch.nn as nn
class MultiTask(transformers.PreTrainedModel):
    def __init__(self, encoder, taskmodels_dict):
        super().__init__(transformers.PretrainedConfig())
        self.encoder = encoder
        self.taskmodels_dict = nn.ModuleDict(taskmodels_dict)

    @classmethod
    def create(cls, model_name, model_type_dict, model_conf_dict):
        shared_encoder = None
        taskmodels_dict = {}

        for task_name, model_type in model_type_dict.items():
            model = model_type.from_pretrained(
                model_name,
                config=model_conf_dict[task_name]
            )

            if shared_encoder is None:
                shared_encoder = getattr(model, cls.get_encoder_attr_name(model))
            else:
                setattr(model, cls.get_encoder_attr_name(model), shared_encoder)
            taskmodels_dict[task_name] = model
        return cls(encoder=shared_encoder, taskmodels_dict=taskmodels_dict)
        
    @classmethod
    def get_encoder_attr_name(cls, model):
        model_class_name = model.__class__.__name__
        if model_class_name.startswith("Bert"):
            return "bert"
        elif model_class_name.startswith("Roberta"):
            return "roberta"

        else:
            raise KeyError(f"Add support for new model {model_class_name}")
        
    def forward(self, task_name, **kwargs):
        return self.taskmodel_dict[task_name](**kwargs)
#test = MultiTask(transformers.BertModel.from_pretrained("bert-base-uncased"), datasets)

In [26]:
model_name = "roberta-base"
#from transformers import BertTokenizer, BertForSequenceClassification
#"cw-train": transformers.
model_type_dict={
        "cw-train": transformers.AutoModelForSequenceClassification,
        "cw-dev": transformers.AutoModelForSequenceClassification,
        "test": transformers.AutoModelForTokenClassification

    }
model_config_dict={
        "cw-train": transformers.AutoConfig.from_pretrained(model_name, num_labels=3),
        "cw-dev": transformers.AutoConfig.from_pretrained(model_name, num_labels=3),
        "test": transformers.AutoConfig.from_pretrained(model_name, num_labels=5)

    }

multitask_model = MultiTask.create(
    model_name,
    model_type_dict,
    model_config_dict
    
)

loading configuration file config.json from cache at /home/chrislindl/.cache/huggingface/hub/models--roberta-base/snapshots/e2da8e2f811d1448a5b465c236feacd80ffbac7b/config.json
Model config RobertaConfig {
  "_name_or_path": "roberta-base",
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2
  },
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "transformers_version": "4.39.3",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 50265


In [27]:
if model_name.startswith("roberta"):
    print(multitask_model.encoder.embeddings.word_embeddings.weight.data_ptr())
    print(multitask_model.taskmodels_dict["cw-train"].roberta.embeddings.word_embeddings.weight.data_ptr())
    print(multitask_model.taskmodels_dict["cw-dev"].roberta.embeddings.word_embeddings.weight.data_ptr())
else:
    print("Exercise for the reader: add a check for other model architectures =)")

140014370148416
140014370148416
140014370148416


In [28]:
tokenizer = transformers.AutoTokenizer.from_pretrained(model_name)

loading configuration file config.json from cache at /home/chrislindl/.cache/huggingface/hub/models--roberta-base/snapshots/e2da8e2f811d1448a5b465c236feacd80ffbac7b/config.json
Model config RobertaConfig {
  "_name_or_path": "roberta-base",
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "transformers_version": "4.39.3",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 50265
}

loading file vocab.json from cache at /home/chrislindl/.cache/huggingface/hub/models--roberta-base/snapshots/e2da8e2f811d1448a5b465c236feacd80ffbac7b/v

In [29]:
max_length = 128

def convert_cw_train_features(batch):
    inputs = batch['Text']
    features = tokenizer.batch_encode_plus(
        inputs,
        max_length=max_length,
        pad_to_max_length=True
    )
    features["labels"] = batch["class_label"]
    return features

convert_func_dict = {
    "cw-train": convert_cw_train_features,
}

In [30]:
columns_dict = {
    "cw-train": ['Sentence_id', 'Text', 'class_label'],
    "cw-dev": ['Sentence_id', 'Text', 'class_label'],
    "test": ['id', 'rumor', 'label', 'timeline', 'evidence']
}

features_dict = {}
for task_name, dataset in datasets.items():
    features_dict[task_name] = {}
    for phase, phase_dataset in dataset.items():
        features_dict[task_name][phase] = phase_dataset.map(
            convert_func_dict[task_name],
            batched=True,
            load_from_cache_file=False,
        )
        print(task_name, phase, len(phase_dataset), len(features_dict[task_name][phase]))
        features_dict[task_name][phase].set_format(
            type='torch', 
            columns=columns_dict[task_name],
        )
        print(task_name, phase, len(phase_dataset), len(features_dict[task_name][phase]))
    break

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/home/chrislindl/dat550-group/venv/lib/python3.8/site-packages/transformers/tokenization_utils_base.py:2688: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


  0%|          | 0/23 [00:00<?, ?it/s]

cw-train train 22501 22501
cw-train train 22501 22501


In [31]:
from transformers import DefaultDataCollator

class NLPDataCollator(DefaultDataCollator):
    def collate_batch(self, features):
        first = features[0]
        if isinstance(first, dict):
            if "labels" in first and first["labels"] is not None:
                if first["labels"].dtype == torch.int64:
                    labels = torch.tensor([f["labels"] for f in features], dtype=torch.long)
                else:
                    labels = torch.tensor([f["labels"] for f in features], dtype=torch.float)
                batch = {"labels": labels}
            for k ,v in first.items():
                if k != "labels":
                    batch[k] = torch.stack([f[k] for f in features])
            return batch
        else:
            return transformers.DefaultDataCollator().collate_batch(features)
    

class StrIgnoreDevice(str):
    def to(self, device):
        return self


In [32]:
class DataLoaderWithTaskname:
    def __init__(self, task_name, dataloader):
        self.task_name = task_name
        self.dataloader = dataloader

        self.batch_size = dataloader.batch_size
        self.dataset = dataloader.dataset

    def __len__(self):
        return len(self.dataloader)
    
    def __iter__(self):
        for batch in self.dataloader:
            batch["task_name"] = StrIgnoreDevice(self.task_name)
            yield batch

class MultitaskDataloader:
    def __init__(self, dataloader_dict):
        self.dataloader_dict = dataloader_dict
        self.num_batches_dict = {
            task_name: len(dataloader)
            for task_name, dataloader in self.dataloader_dict.items()
        }
        self.task_name_list = list(self.dataloader_dict)
        self.dataset = [None] * sum(
            len(dataloader.dataset)
            for dataloader in self.dataloader_dict.values()
        )

    def __len__(self):
        return sum(self.num_batches_dict.values())
    
    def __iter__(self):
        task_choice_list = []
        for i, task_name in enumerate(self.task_name_list):
            task_choice_list += [i] * self.num_batches_dict[task_name]
        task_choice_list = np.array(task_choice_list)
        np.random.shuffle(task_choice_list)
        dataloader_iter_dict = {
            task_name: iter(dataloader)
            for task_name, dataloader in self.dataloader_dict.items()
        }
        for task_choice in task_choice_list:
            task_name = self.task_name_list[task_choice]
            yield next(dataloader_iter_dict[task_name])


In [42]:
from transformers import Trainer
#from transformers.data.data_collator import torch_default_data_collator
from transformers.trainer_utils import is_tf_available
from transformers.trainer import get_dataloader_sampler
#from accelerate.state import is_tf_available
from torch.utils.data.distributed import DistributedSampler
from torch.utils.data.sampler import RandomSampler
from torch.utils.data.dataloader import DataLoader
class MultitaskTrainer(Trainer):
    def get_single_train_dataloader(self, task_name, train_dataset):
        if self.train_dataset is None:
            raise ValueError("Trainer: training requires a train_dataset.")
        if is_tf_available():
            train_sampler = get_dataloader_sampler(train_dataset)
        else:
            train_sampler = (
                RandomSampler(train_dataset)
                if self.args.local_rank == -1
                else DistributedSampler(train_dataset)
            )
        data_loader = DataLoaderWithTaskname(
            task_name=task_name,
            data_loader=DataLoader(
                train_dataset,
                batch_size=self.args.train_batch_size,
                sampler=train_sampler,
                collate_fn=self.data_collator.collate_batch,
            ),
        )

        if is_tf_available():
            data_loader = nn.pl.ParallelLoader(data_loader, [self.args.device]).per_device_loader(self.args.device)
        return data_loader
    
    def get_train_dataloader(self):
        loader = MultitaskDataloader({
            task_name: self.get_single_train_dataloader(task_name, dataset)
            for task_name, dataset in self.train_dataset.items()
        })
        return loader
            

    

In [43]:
train_dataset = {
    task_name: dataset["train"]
    for task_name, dataset in features_dict.items()
}
trainer = MultitaskTrainer(
    model=multitask_model,
    args=transformers.TrainingArguments(
        output_dir="models/multitask_model",
        overwrite_output_dir=True,
        learning_rate=1e-5,
        do_train=True,
        num_train_epochs=3,
        per_device_train_batch_size=8,
        save_steps=3000,
    ),
    data_collator=NLPDataCollator(),
    train_dataset=train_dataset,
)


PyTorch: setting up devices
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
/home/chrislindl/dat550-group/venv/lib/python3.8/site-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
datalo

In [44]:
trainer.train()

ValueError: Default process group has not been initialized, please make sure to call init_process_group.